# Problem Statement

A small engineering consulting firm has 3 senior designers available to work on the firm's 4 current projects over the next 2 weeks. Each designer has 80 hours to split among the projects, and the following table shows the manager's scoring $(0=$ nil to $100=$ perfect $)$ of the capability of each designer to contribute to each project, along with his estimate of the hours that each project will require.


|      Designer     | Project 1    | Project 2    | Project 3    | Project 4    |
|----------|--------------|--------------|--------------|--------------|
| 1        | 90           | 80           | 10           | 50           |
| 2        | 60           | 70           | 50           | 65           |
| 3        | 70           | 40           | 80           | 85           |



|     **Required:**      | Project 1    | Project 2    | Project 3    | Project 4    |
|-----------|--------------|--------------|--------------|--------------|
| **Hours** | 70           | 50           | 85           | 35           |


#### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import shutil
import sys
import os.path
from pyomo.environ import *

import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

#### Defining Data

In [ ]:
Engg = {'E1','E2','E3'}    # Check the type by: type(Engg)

In [ ]:
Proj = {'P1','P2','P3','P4'}

In [ ]:
score = {
    ('E1','P1'):90,
    ('E1','P2'):80,
    ('E1','P3'):10,
    ('E1','P4'):50,
    ('E2','P1'):60,
    ('E2','P2'):70,
    ('E2','P3'):50,
    ('E2','P4'):65,
    ('E3','P1'):70,
    ('E3','P2'):40,
    ('E3','P3'):80,
    ('E3','P4'):85,
}   # Dictionary with tuples as keys (based on our defined sets)

In [ ]:
hours_needed = {
    ('P1'):70,
    ('P2'):50,
    ('P3'):85,
    ('P4'):35,
}

In [ ]:
max_hours = 80

## Model

Let the design engineers be set $E$ with $E_{i} \; :i \in [1,2,3]$ and the projects be $P$ with  $P_{j} \; :j \in [1,2,3,4]$. We can model the problem as allocation of the number of hours $H_{ij}$ with each design engineer $E_{i}$ $\forall i $ that are being put onto the projects $P_{j}$ $\forall j $, given the $i^{th}$ engineer $E$ works on $j^{th}$ project with given score $e_{ij}$.

Let the maximum hours available with each engineer be $H_{max}$ and the required number of hours for each project $P_j$ be $R_{j} \; \forall j $. Thus, then the mathematical formulation can be made as:

$$
\text{Maximize } \sum_{i \in E} \sum_{j \in P} H_{ij} e_{ij}
$$

Subject to:

$$
\sum_{j \in P} H_{ij} \leq H_{max} \;\; \forall i \in E
$$

$$
\sum_{i \in E} H_{ij} \geq R_j \;\; \forall j \in P
$$

$$
H_{ij} \geq 0 \;\; \forall i,j
$$


## Implement

In [ ]:
m = pe.ConcreteModel()

#### Initializing Sets

In [ ]:
m.Engg = pe.Set(initialize=Engg)
m.Proj = pe.Set(initialize=Proj)     # Ignore if any warnings appear

#### Initializing Parameters

In [ ]:
m.score = pe.Param(m.Engg, m.Proj, initialize=score)
m.hours_needed = pe.Param(m.Proj, initialize=hours_needed)
m.max_hours = pe.Param(initialize=max_hours)

#### Initializing [Variables](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html)
> Note: Refer the documnetation to also note other Predefined Virtual Sets

In [ ]:
m.H = pe.Var(m.Engg, m.Proj, domain=pe.NonNegativeReals)


#### Defining Objective

In [ ]:
obj_expr = sum(m.H[i,j]*m.score[i,j]
               for i in m.Engg for j in m.Proj)
m.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

#### Defining [Constraints](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html)
> Note: Refer the documentation to see other ways to write the constraints, such as using a Constraint List

In [ ]:
def maxhour_rule(m,i):
    return sum(m.H[i,j] for j in m.Proj) <= m.max_hours

m.maximum_hours =  pe.Constraint(m.Engg, rule=maxhour_rule)

In [ ]:
def projhour_rule(m,j):
    return sum(m.H[i,j] for i in m.Engg) >= m.hours_needed[j]

m.proj_hours = pe.Constraint(m.Proj, rule=projhour_rule)

## Solve and Postprocess

In [ ]:
solver = po.SolverFactory('glpk')
results = solver.solve(m, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpgw6e59zd.glpk.raw --wglp /tmp/tmpy1f36lnn.glpk.glp --cpxlp
 /tmp/tmpx3iu_sq0.pyomo.lp
Reading problem data from '/tmp/tmpx3iu_sq0.pyomo.lp'...
7 rows, 12 columns, 24 non-zeros
78 lines were read
Writing problem data to '/tmp/tmpy1f36lnn.glpk.glp'...
65 lines were written
GLPK Simplex Optimizer 5.0
7 rows, 12 columns, 24 non-zeros
Preprocessing...
7 rows, 12 columns, 24 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
      0: obj =  -0.000000000e+00 inf =   2.400e+02 (4)
      6: obj =   1.062500000e+04 inf =   0.000e+00 (0)
*    10: obj =   1.882500000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/tmpgw6e59zd.glpk.raw'...
28 lines were written


In [ ]:
print("Optimal Assignment Objective: ",pe.value(m.obj))

Optimal Assignment Objective:  18825.0


In [ ]:
for i in m.Engg:
    for j in m.Proj:
        print("Engg",i," working on Project",j,"= ",pe.value(m.H[i,j])," Hours")


Engg E3  working on Project P4 =  0.0  Hours
Engg E3  working on Project P2 =  0.0  Hours
Engg E3  working on Project P3 =  80.0  Hours
Engg E3  working on Project P1 =  0.0  Hours
Engg E1  working on Project P4 =  0.0  Hours
Engg E1  working on Project P2 =  10.0  Hours
Engg E1  working on Project P3 =  0.0  Hours
Engg E1  working on Project P1 =  70.0  Hours
Engg E2  working on Project P4 =  35.0  Hours
Engg E2  working on Project P2 =  40.0  Hours
Engg E2  working on Project P3 =  5.0  Hours
Engg E2  working on Project P1 =  0.0  Hours


In [ ]:
for i in m.Engg:
    h = pe.value(sum(m.H[i,j] for j in m.Proj))
    print("Hours Worked by Engineer",i,"= ",h," Hours")

Hours Worked by Engineer E3 =  80.0  Hours
Hours Worked by Engineer E1 =  80.0  Hours
Hours Worked by Engineer E2 =  80.0  Hours
